 Data Dictionary – SciKey UPEC Dataset

upec_publications`

| Column                  | Type      | Description                                                                 |
|-------------------------|-----------|-----------------------------------------------------------------------------|
| **docid**               | Integer   | Unique numeric ID of the publication in HAL (primary key).                  |
| **halId_s**             | String    | Official HAL identifier (e.g., `hal-01234567`).                             |
| **doiId_s**             | String    | DOI (Digital Object Identifier) if available; empty if missing.             |
| **citationRef_s**       | String    | Formatted citation text (authors, title, venue, year).                      |
| **title_s**             | Text      | Title of the publication.                                                   |
| **abstract_s**          | Text      | Abstract of the publication (may be missing for some).                       |
| **journalTitle_s**      | String    | Journal name (if it’s a journal article; may be missing).                   |
| **journalIssn_s**       | String    | Journal ISSN (unique identifier for journals; may be missing).              |
| **producedDate_tdate**  | Date      | Official production/publication date of the work.                           |
| **submittedDate_tdate** | Date      | Date when the work was deposited into HAL (repository submission date).     |
| **language_s**          | String    | Language code of the publication (`en` in your dataset).                     |

`upec_authors`
Each row = one author of one publication.

| Column              | Type      | Description                                                                 |
|---------------------|-----------|-----------------------------------------------------------------------------|
| **docid**           | Integer   | Publication ID (foreign key → links to `upec_publications.docid`).          |
| **author_position** | Integer   | Position of the author in the publication’s author list (1 = first author). |
| **name_full**       | String    | Full name of the author (as recorded in HAL).                               |

Key Points

- **One-to-many relationship**:  
  - One `docid` in `upec_publications` can have many rows in `upec_authors`.  
- **Author order preserved**:  
  - `author_position` keeps track of who is first, second, … last author.  
- **Usage examples**:  
  - Find all publications by a given author.  
  - Extract first-author papers only.  
  - Count the average number of authors per paper.  

 Example

### Publications Table (`upec_publications`)
| docid   | title_s                        | producedDate_tdate | journalTitle_s   |
|---------|-------------------------------|--------------------|------------------|
| 1000798 | *A study on number theory*    | 2019-05-10         | J. Math. Res.    |
| 1001810 | *Advances in civil materials* | 2021-03-12         | Materials Today  |

### Authors Table (`upec_authors`)
| docid   | author_position | name_full        |
|---------|-----------------|------------------|
| 1000798 | 1               | Yann Bugeaud     |
| 1000798 | 2               | Lingmin Liao     |
| 1001810 | 1               | C. Caggegi       |
| 1001810 | 2               | Vincent Pensée   |
| 1001810 | 3               | M. Fagone        |





Install necessary Libraries 

In [ ]:
!pip install requests pandas tqdm tenacity

In [ ]:
import time, json
import pandas as pd
import requests
from urllib.parse import urlencode
from tenacity import retry, wait_exponential, stop_after_attempt
from tqdm import tqdm

pd.set_option("display.max_columns", 80)
pd.set_option("display.width", 180)


Configure:
- **HAL portal** → UPEC (`u-pec`)
- **Filters** → English only (`language_s:en`) — remove or change later as needed
- **Fields** → include reference IDs (docid, halId_s, doiId_s, citationRef_s), text, date, and language
- **Outputs** → where files will be saved


In [ ]:
#==HAL portal (UPEC) ==
HAL_PORTAL = "u-pec"        

#==Page size==
ROWS_PER_PAGE = 500 

#==Filters== #==English only
FILTERS = ["language_s:en"]

#==Fields==
FIELDS = ",".join([
    "docid",                # internal HAL numeric ID
    "halId_s",              # official HAL ID (e.g., hal-01234567)
    "doiId_s",              # DOI if available
    "citationRef_s",        # formatted citation string
    "title_s",
    "abstract_s",
    "authFullName_s",       # authors (list)
    "producedDate_tdate",   # publication/production date
    "submittedDate_tdate",  # date submitted to HAL (useful, optional)
    "journalTitle_s",       # journal name (if article)
    "journalIssn_s",        # journal ISSN (optional)
    "language_s"
])

#==Outputs==
CSV_ALL = "upec_hal_all_en.csv"
CSV_WITH_ABS = "upec_hal_with_abstracts_en.csv"
XLSX_ALL = "upec_hal_all_en.xlsx"

#==Base endpoint scoped to portal==
BASE = f"https://api.archives-ouvertes.fr/search/{HAL_PORTAL}/"


function to fetch **one page** of results using **cursorMark** paging (the safe way to retrieve many records).  
It also retries automatically if there’s a temporary network hiccup.


In [1]:
@retry(wait=wait_exponential(multiplier=1, min=1, max=60),
       stop=stop_after_attempt(6))
def fetch_page(cursor="*"):
    params = {
        "q": "*:*",
        "wt": "json",
        "fl": FIELDS,
        "rows": ROWS_PER_PAGE,
        "sort": "docid asc",
        "cursorMark": cursor
    }
    if FILTERS:
        params["fq"] = FILTERS
    url = f"{BASE}?{urlencode(params, doseq=True)}"
    print(f"Fetching: {url}")
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    return r.json()


NameError: name 'retry' is not defined

using HAL to check how many results match the query/filters **without** downloading them, to sanity-check the scope.


In [9]:
params = {"q":"*:*", "wt":"json", "rows":0}
if FILTERS: params["fq"] = FILTERS
probe = requests.get(f"{BASE}?{urlencode(params, doseq=True)}", timeout=60).json()
print("Estimated total (numFound):", probe.get("response", {}).get("numFound"))


Estimated total (numFound): 31382


**Downloading upec Publications  from HAL**

In [11]:
all_rows = []
cursor = "*"

with tqdm(desc="Downloading from HAL", unit="docs") as bar:
    while True:
        data = fetch_page(cursor)
        docs = data.get("response", {}).get("docs", [])
        if not docs:
            break

        for d in docs:
            # Flatten single-value lists
            for key in [
                "title_s", "abstract_s", "halId_s", "doiId_s", "citationRef_s", "language_s",
                "producedDate_tdate", "submittedDate_tdate", "journalTitle_s", "journalIssn_s"
            ]:
                if isinstance(d.get(key), list) and d[key]:
                    d[key] = d[key][0]

            # Authors: join list by semicolon
            if isinstance(d.get("authFullName_s"), list):
                d["authFullName_s"] = "; ".join(d["authFullName_s"])

            all_rows.append(d)

        bar.update(len(docs))
        next_cursor = data.get("nextCursorMark")
        if not next_cursor or next_cursor == cursor:
            break
        cursor = next_cursor
        time.sleep(0.2)  # be gentle to the API

df = pd.DataFrame(all_rows)
print("Rows downloaded:", len(df))
df.head()


Rows downloaded: 31382


,docid,citationRef_s,title_s,abstract_s,journalTitle_s,journalIssn_s,authFullName_s,language_s,halId_s,doiId_s,submittedDate_tdate,producedDate_tdate
0,1000798,"<i>Ergodic Theory and Dynamical Systems</i>, 2...",Uniform Diophantine approximation related to $...,Let $b\geq 2$ be an integer and $\hv$ a real n...,Ergodic Theory and Dynamical Systems,0143-3857,Yann Bugeaud; Lingmin Liao,en,hal-00975111,10.1017/etds.2014.66,2014-06-04T13:36:40Z,2016-02-01T00:00:00Z
1,1001810,"<i>Construction and Building Materials</i>, 20...",Experimental global analysis of the efficiency...,The study focuses on the improvement of the co...,Construction and Building Materials,0950-0618,C. Caggegi; Vincent Pensée; M. Fagone; M. Cuom...,en,hal-01001810,10.1016/j.conbuildmat.2013.11.086,2014-06-04T21:18:14Z,2014-02-28T00:00:00Z
2,1002129,"<i>American Journal of Kidney Diseases</i>, 20...",APOL1 polymorphisms and development of CKD in ...,We report an occurrence of progressive loss of...,American Journal of Kidney Diseases,0272-6386,Tomek Kofman; Vincent Audard; Céline Narjoz; O...,en,inserm-01002129,10.1053/j.ajkd.2013.12.014,2014-06-05T16:16:04Z,2014-05-01T00:00:00Z
3,1002652,<i>ICWRS 2014 : Evolving Water Resources Syste...,Urban stormwater source control policies: why ...,Stormwater source control is becoming a common...,NaN,NaN,Guido Petrucci; José-Frédéric Deroubaix; Bruno...,en,hal-01002652,NaN,2014-06-06T14:54:34Z,2014-06-04T00:00:00Z
4,1004672,<i>2013 EUMETSAT Meteorological Satellite Conf...,The IASI-NG mission onboard METOP-SG: Scientif...,Thermal infrared sounders onboard polar-orbiti...,NaN,NaN,Cyril Crevoisier; Cathy Clerbaux; Vincent Guid...,en,hal-01004672,NaN,2014-06-11T15:33:26Z,2013-09-16T00:00:00Z


Split the `authFullName_s` string into separate columns (`Author_1`, `Author_2`, …) and put them **at the end**.  
Column order becomes: references → text → date → language → authors.


In [13]:
# Split authors into Author_1..N columns
if "authFullName_s" in df.columns:
    authors_split = df["authFullName_s"].fillna("").str.split(";", expand=True)
    # Use .map instead of applymap to avoid FutureWarning
    authors_split = authors_split.map(lambda x: x.strip() if isinstance(x, str) else x)
    authors_split.columns = [f"Author_{i+1}" for i in range(authors_split.shape[1])]
    df_final = pd.concat([df.drop(columns=["authFullName_s"]), authors_split], axis=1)
else:
    df_final = df.copy()

# Reorder columns
author_cols = [c for c in df_final.columns if c.startswith("Author_")]
front = [c for c in [
    "docid","halId_s","doiId_s","citationRef_s",
    "title_s","abstract_s",
    "journalTitle_s","journalIssn_s",
    "producedDate_tdate","submittedDate_tdate",
    "language_s"
] if c in df_final.columns]

df_final = df_final[ front + author_cols ]

df_final.head()



,docid,halId_s,doiId_s,citationRef_s,title_s,abstract_s,journalTitle_s,journalIssn_s,producedDate_tdate,submittedDate_tdate,language_s,Author_1,Author_2,Author_3,Author_4,Author_5,Author_6,Author_7,Author_8,Author_9,Author_10,Author_11,Author_12,Author_13,Author_14,Author_15,Author_16,Author_17,Author_18,Author_19,Author_20,Author_21,Author_22,Author_23,Author_24,Author_25,Author_26,Author_27,Author_28,Author_29,...,Author_1742,Author_1743,Author_1744,Author_1745,Author_1746,Author_1747,Author_1748,Author_1749,Author_1750,Author_1751,Author_1752,Author_1753,Author_1754,Author_1755,Author_1756,Author_1757,Author_1758,Author_1759,Author_1760,Author_1761,Author_1762,Author_1763,Author_1764,Author_1765,Author_1766,Author_1767,Author_1768,Author_1769,Author_1770,Author_1771,Author_1772,Author_1773,Author_1774,Author_1775,Author_1776,Author_1777,Author_1778,Author_1779,Author_1780,Author_1781
0,1000798,hal-00975111,10.1017/etds.2014.66,"<i>Ergodic Theory and Dynamical Systems</i>, 2...",Uniform Diophantine approximation related to $...,Let $b\geq 2$ be an integer and $\hv$ a real n...,Ergodic Theory and Dynamical Systems,0143-3857,2016-02-01T00:00:00Z,2014-06-04T13:36:40Z,en,Yann Bugeaud,Lingmin Liao,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1001810,hal-01001810,10.1016/j.conbuildmat.2013.11.086,"<i>Construction and Building Materials</i>, 20...",Experimental global analysis of the efficiency...,The study focuses on the improvement of the co...,Construction and Building Materials,0950-0618,2014-02-28T00:00:00Z,2014-06-04T21:18:14Z,en,C. Caggegi,Vincent Pensée,M. Fagone,M. Cuomo,Luc Chevalier,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1002129,inserm-01002129,10.1053/j.ajkd.2013.12.014,"<i>American Journal of Kidney Diseases</i>, 20...",APOL1 polymorphisms and development of CKD in ...,We report an occurrence of progressive loss of...,American Journal of Kidney Diseases,0272-6386,2014-05-01T00:00:00Z,2014-06-05T16:16:04Z,en,Tomek Kofman,Vincent Audard,Céline Narjoz,Olivier Gribouval,Marie Matignon,Claire Leibler,Dominique Desvaux,Philippe Lang,Philippe Grimbert,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1002652,hal-01002652,NaN,<i>ICWRS 2014 : Evolving Water Resources Syste...,Urban stormwater source control policies: why ...,Stormwater source control is becoming a common...,NaN,NaN,2014-06-04T00:00:00Z,2014-06-06T14:54:34Z,en,Guido Petrucci,José-Frédéric Deroubaix,Bruno Tassin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1004672,hal-01004672,NaN,<i>2013 EUMETSAT Meteorological Satellite Conf...,The IASI-NG mission onboard METOP-SG: Scientif...,Thermal infrared sounders onboard polar-orbiti...,NaN,NaN,2013-09-16T00:00:00Z,2014-06-11T15:33:26Z,en,Cyril Crevoisier,Cathy Clerbaux,Vincent Guidard,Thierry Phulpin,Raymond Armante,Brice Barret,Claude Camy-Peyret,Jean-Pierre Chaboureau,Gaëlle Dufour,Lydie Lavanant,Juliette Hadji

Saving the various csv files 

**upec_hal_all_en.csv** → full dataset with Author_1…N (backup).

**upec_hal_with_abstracts_en.csv** → abstracts-only subset.

**upec_hal_all_en.xlsx** → Excel version of the full dataset.

**upec_publications.csv** → clean publications table (no Author_N columns).

**upec_hal_authors_long_en.csv** → normalized author-publication pairs


In [15]:


import re
import pandas as pd
from pandas.api.types import is_object_dtype

# 1) Sanitizers
_ILLEGAL_CTRL_RE = re.compile(r'[\x00-\x08\x0B-\x0C\x0E-\x1F]')   # Excel-banned control chars
_HTML_TAG_RE     = re.compile(r'<[^>]+>')                         # strip simple HTML tags

def sanitize_for_excel(val):
    if pd.isna(val):
        return val
    s = str(val)
    s = _HTML_TAG_RE.sub('', s)           # remove <i>…</i>, etc.
    s = _ILLEGAL_CTRL_RE.sub('', s)       # remove hidden control chars
    s = s.replace('\u00AD', '-')          # soft hyphen → -
    s = s.replace('\u2011', '-')          # non-breaking hyphen → -
    if len(s) > 32767:                    # Excel cell limit
        s = s[:32767]
    return s

# 2) Apply sanitizer only to string-like columns
df_final_clean = df_final.copy()
for col in df_final_clean.columns:
    if is_object_dtype(df_final_clean[col]):
        df_final_clean[col] = df_final_clean[col].map(sanitize_for_excel)

# 3) Save: Full EN dataset (CSV)
df_final_clean.to_csv(CSV_ALL, index=False, encoding="utf-8")
print("Saved:", CSV_ALL)

# 4) Save: Abstracts-only EN dataset (CSV)
if "abstract_s" in df_final_clean.columns:
    df_with_abs = df_final_clean[df_final_clean["abstract_s"].notna()].copy()
else:
    df_with_abs = df_final_clean.copy()
df_with_abs.to_csv(CSV_WITH_ABS, index=False, encoding="utf-8")
print("Saved:", CSV_WITH_ABS, " (rows:", len(df_with_abs), ")")

# 5) Save: Excel (full EN)
try:
    df_final_clean.to_excel(XLSX_ALL, index=False)
    print("Saved:", XLSX_ALL)
except Exception as e:
    print("Excel save failed. You can still use the CSVs above.")
    print("Error:", e)

# 6) Create publications table (drop Author_1…N)
author_cols = [c for c in df_final_clean.columns if c.startswith("Author_")]
df_publications = df_final_clean.drop(columns=author_cols, errors="ignore")
df_publications.to_csv("upec_publications.csv", index=False, encoding="utf-8")
print("Saved:", "upec_publications.csv (rows:", len(df_publications), ")")

# 7) Create normalized authors table (long format)
rows = []
for _, r in df_final_clean.iterrows():
    pub_id = r.get("docid")
    for i, c in enumerate(author_cols, start=1):
        name = r.get(c)
        if isinstance(name, str) and name.strip():
            rows.append({
                "docid": pub_id,
                "author_position": i,
                "name_full": name.strip()
            })
df_auth_long = pd.DataFrame(rows)
df_auth_long.to_csv("upec_hal_authors_long_en.csv", index=False, encoding="utf-8")
print("Saved:", "upec_hal_authors_long_en.csv (rows:", len(df_auth_long), ")")


Saved: upec_hal_all_en.csv
Saved: upec_hal_with_abstracts_en.csv  (rows: 20439 )
Saved: upec_hal_all_en.xlsx
Saved: upec_publications.csv (rows: 31382 )
Saved: upec_hal_authors_long_en.csv (rows: 252663 )


 summary 


In [17]:
def miss(col):
    """Count how many rows are missing for a given column."""
    return int(df_final[col].isna().sum()) if col in df_final.columns else "N/A"

total = len(df_final)
author_cols = [c for c in df_final.columns if c.startswith("Author_")]

print("=== SciKey UPEC Dataset Summary (English) ===")
print("Total rows:", total)

# Core text fields
print("Titles missing:", miss("title_s"))
print("Abstracts missing:", miss("abstract_s"))

# Dates
print("Produced dates missing (official publication date):", miss("producedDate_tdate"))
print("Submitted dates missing (deposit in HAL):", miss("submittedDate_tdate"))

# Language distribution
if "language_s" in df_final.columns:
    print("\nLanguage breakdown (should all be 'en'):")
    print(df_final["language_s"].value_counts(dropna=False).to_string())

# References
print("\nHAL IDs missing:", miss("halId_s"))
print("DOIs missing:", miss("doiId_s"))
print("Citation references missing:", miss("citationRef_s"))

# Journal info
print("\nJournal titles missing:", miss("journalTitle_s"))
print("Journal ISSN missing:", miss("journalIssn_s"))

# Authors
print("\nRows with all author columns empty:",
      df_final[author_cols].isna().all(axis=1).sum() if author_cols else "N/A")


=== SciKey UPEC Dataset Summary (English) ===
Total rows: 31382
Titles missing: 0
Abstracts missing: 10943
Produced dates missing (official publication date): 0
Submitted dates missing (deposit in HAL): 0

Language breakdown (should all be 'en'):
language_s
en    31382

HAL IDs missing: 0
DOIs missing: 12047
Citation references missing: 0

Journal titles missing: 11446
Journal ISSN missing: 12407

Rows with all author columns empty: 0


 Normalized authors (long format) for database loads. Create a **long** authors table where each row is (docid, author_position, name_full).  
This solves “variable number of authors” and is easy to load into a relational database.


In [32]:
# === Normalized (long) author table ===

rows = []
author_cols = [c for c in df_final.columns if c.startswith("Author_")]

for _, r in df_final.iterrows():
    pub_id = r.get("docid")
    for i, c in enumerate(author_cols, start=1):
        name = r.get(c)
        if isinstance(name, str) and name.strip():
            rows.append({
                "docid": pub_id,
                "author_position": i,
                "name_full": name.strip()
            })

df_auth_long = pd.DataFrame(rows)

# Save to CSV for DB import
df_auth_long.to_csv("upec_hal_authors_long_en.csv", index=False, encoding="utf-8")

print("Saved normalized author table: upec_hal_authors_long_en.csv")
print("Rows:", len(df_auth_long))
print("Unique publications:", df_auth_long['docid'].nunique())
print("Max authors per publication:", df_auth_long['author_position'].max())
df_auth_long.head()


Saved normalized author table: upec_hal_authors_long_en.csv
Rows: 252663
Unique publications: 31382
Max authors per publication: 1781


,docid,author_position,name_full
0,1000798,1,Yann Bugeaud
1,1000798,2,Lingmin Liao
2,1001810,1,C. Caggegi
3,1001810,2,Vincent Pensée
4,1001810,3,M. Fagone
